## Library import 

In [196]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import sys
from urllib.error import HTTPError

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [197]:
players_boxscores = pd.read_csv('./pipeline_output/nba_players_boxscores_2022-09-08.csv')
players_boxscores.shape
# # test.head()

(597659, 32)

In [198]:
players_boxscores.head()

,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
0,2001,AC Green,1,1,2000-11-01,37-028,MIA,NaN,ORL,W (+26),0,14:53,2,4,0.500,0,0,NaN,1,2,0.5,1,3,4,0,0,1,1,2,5,3.1,15.0
1,2001,AC Green,2,2,2000-11-03,37-030,MIA,@,CHH,L (-4),0,7:08,0,1,0.000,0,0,NaN,0,0,NaN,0,1,1,0,0,0,3,1,0,-3.8,-10.0
2,2001,AC Green,3,3,2000-11-04,37-031,MIA,NaN,PHI,L (-2),0,5:37,1,3,0.333,0,0,NaN,0,0,NaN,1,0,1,0,0,0,0,2,2,0.2,4.0
3,2001,AC Green,4,4,2000-11-08,37-035,MIA,NaN,SEA,W (+6),0,13:21,0,3,0.000,0,0,NaN,0,0,NaN,0,1,1,0,0,0,0,2,0,-2.6,-7.0
4,2001,AC Green,5,5,2000-11-10,37-037,MIA,@,UTA,W (+7),0,15:08,0,2,0.000,0,0,NaN,0,0,NaN,2,3,5,0,0,1,1,2,0,-0.2,11.0


In [199]:
players_boxscores[players_boxscores['player_name'].str.contains('LeBron James')].head()

,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
358134,2004,LeBron James,1,1,2003-10-29,18-303,CLE,@,SAC,L (-14),1,42:50,12,20,0.600,0,2,0.0,1,3,0.333,2,4,6,9,4,0,2,3,25,24.7,-9.0
358135,2004,LeBron James,2,2,2003-10-30,18-304,CLE,@,PHO,L (-9),1,40:21,8,17,0.471,1,5,0.2,4,7,0.571,2,10,12,8,1,0,7,1,21,14.7,-3.0
358136,2004,LeBron James,3,3,2003-11-01,18-306,CLE,@,POR,L (-19),1,39:10,3,12,0.250,0,1,0.0,2,2,1.000,0,4,4,6,2,0,2,3,8,5.0,-21.0
358137,2004,LeBron James,4,4,2003-11-05,18-310,CLE,NaN,DEN,L (-4),1,41:06,3,11,0.273,0,2,0.0,1,1,1.000,2,9,11,7,2,3,2,1,7,11.2,-3.0
358138,2004,LeBron James,5,5,2003-11-07,18-312,CLE,@,IND,L (-1),1,43:44,8,18,0.444,1,2,0.5,6,7,0.857,0,5,5,3,0,0,7,2,23,9.0,-7.0


In [200]:
players_boxscores[players_boxscores['player_name'].str.contains('Dwyane Wade')].head()

,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
174822,2004,Dwyane Wade,1,1,2003-10-28,21-284,MIA,@,PHI,L (-15),1,41:02,8,18,0.444,0,1,0.0,2,3,0.667,1,3,4,4,0,0,5,3,18,6.4,-5.0
174823,2004,Dwyane Wade,2,2,2003-10-29,21-285,MIA,@,BOS,L (-23),1,33:32,2,12,0.167,1,2,0.5,0,0,NaN,1,1,2,4,1,0,0,5,5,0.2,-17.0
174824,2004,Dwyane Wade,3,3,2003-10-31,21-287,MIA,NaN,DET,L (-12),1,23:41,1,9,0.111,0,0,NaN,2,2,1.000,2,1,3,2,0,1,2,1,4,-0.5,-17.0
174825,2004,Dwyane Wade,6,4,2003-11-08,21-295,MIA,NaN,MIN,L (-9),1,42:00,8,21,0.381,1,2,0.5,2,4,0.500,0,3,3,5,1,1,3,3,19,8.6,-6.0
174826,2004,Dwyane Wade,7,5,2003-11-11,21-298,MIA,@,HOU,L (-20),1,37:54,4,9,0.444,0,1,0.0,3,4,0.750,0,5,5,5,3,0,3,3,11,9.7,-15.0


In [201]:
players_boxscores[players_boxscores['player_name'].str.contains("Shaquille ONeal")].head()


,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
513437,2001,Shaquille ONeal,1,1,2000-10-31,28-239,LAL,@,POR,W (+10),1,43:21,14,20,0.700,0,0,NaN,8,16,0.500,3,8,11,5,1,4,1,1,36,34.8,7.0
513438,2001,Shaquille ONeal,2,2,2000-11-01,28-240,LAL,NaN,UTA,L (-5),1,43:21,14,20,0.700,0,0,NaN,6,18,0.333,5,10,15,1,2,0,4,6,34,23.6,2.0
513439,2001,Shaquille ONeal,3,3,2000-11-04,28-243,LAL,@,VAN,W (+9),1,40:56,12,20,0.600,0,1,0.0,3,14,0.214,3,12,15,5,1,3,1,1,27,24.3,20.0
513440,2001,Shaquille ONeal,4,4,2000-11-05,28-244,LAL,NaN,LAC,W (+5),1,42:31,15,31,0.484,0,0,NaN,9,18,0.500,4,10,14,4,0,5,0,5,39,29.8,13.0
513441,2001,Shaquille ONeal,5,5,2000-11-07,28-246,LAL,@,HOU,L (-10),1,45:01,8,19,0.421,0,0,NaN,8,14,0.571,1,6,7,5,0,3,4,3,24,14.4,-6.0


# ----------------------------------------------------------------------------
# PART 3: Players Attributes
# ----------------------------------------------------------------------------

In [202]:
players_attributes = pd.read_csv('./pipeline_output/nba_list_players_multi_season_dataset_2022-08-31.csv')
players_attributes.shape

(11952, 10)

In [203]:
players_attributes.head(10)

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College
0,2001,ATL,Cal Bowdler,SF,6-10,245,"March 31, 1977",us,1,Old Dominion
1,2001,ATL,Ira Bowman,PG,6-5,195,"June 11, 1973",us,1,"Providence, Penn"
2,2001,ATL,Sean Colson,PG,6-0,175,"July 1, 1975",us,R,"Rhode Island, UNC Charlotte"
3,2001,ATL,Chris Crawford,PF,6-9,235,"May 13, 1975",us,3,Marquette
4,2001,ATL,Dion Glover,SG,6-5,228,"October 22, 1978",us,1,Georgia Tech
5,2001,ATL,Alan Henderson,PF,6-9,235,"December 2, 1972",us,5,Indiana
6,2001,ATL,Jim Jackson,SF,6-6,220,"October 14, 1970",us,8,Ohio State
7,2001,ATL,Anthony Johnson,PG,6-3,190,"October 2, 1974",us,3,College of Charleston
8,2001,ATL,DerMarr Johnson,SF,6-9,201,"May 5, 1980",us,R,Cincinnati
9,2001,ATL,Brevin Knight,PG,5-10,173,"November 8, 1975",us,3,Stanford


In [204]:
players_attributes['Name'] = players_attributes['Name'].str.replace("'", '', regex=True)
players_attributes['Name'] = players_attributes['Name'].str.replace("-", '', regex=True)
players_attributes['Name'] = players_attributes['Name'].str.replace(".", '', regex=True)
players_attributes['Name'] = players_attributes['Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [205]:
players_attributes.head(10)

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College
0,2001,ATL,Cal Bowdler,SF,6-10,245,"March 31, 1977",us,1,Old Dominion
1,2001,ATL,Ira Bowman,PG,6-5,195,"June 11, 1973",us,1,"Providence, Penn"
2,2001,ATL,Sean Colson,PG,6-0,175,"July 1, 1975",us,R,"Rhode Island, UNC Charlotte"
3,2001,ATL,Chris Crawford,PF,6-9,235,"May 13, 1975",us,3,Marquette
4,2001,ATL,Dion Glover,SG,6-5,228,"October 22, 1978",us,1,Georgia Tech
5,2001,ATL,Alan Henderson,PF,6-9,235,"December 2, 1972",us,5,Indiana
6,2001,ATL,Jim Jackson,SF,6-6,220,"October 14, 1970",us,8,Ohio State
7,2001,ATL,Anthony Johnson,PG,6-3,190,"October 2, 1974",us,3,College of Charleston
8,2001,ATL,DerMarr Johnson,SF,6-9,201,"May 5, 1980",us,R,Cincinnati
9,2001,ATL,Brevin Knight,PG,5-10,173,"November 8, 1975",us,3,Stanford


In [206]:
# Birth date as date
players_attributes['BirthDate'] = pd.to_datetime(players_attributes['BirthDate'], format="%B %d, %Y")
players_attributes.head(10)

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College
0,2001,ATL,Cal Bowdler,SF,6-10,245,1977-03-31,us,1,Old Dominion
1,2001,ATL,Ira Bowman,PG,6-5,195,1973-06-11,us,1,"Providence, Penn"
2,2001,ATL,Sean Colson,PG,6-0,175,1975-07-01,us,R,"Rhode Island, UNC Charlotte"
3,2001,ATL,Chris Crawford,PF,6-9,235,1975-05-13,us,3,Marquette
4,2001,ATL,Dion Glover,SG,6-5,228,1978-10-22,us,1,Georgia Tech
5,2001,ATL,Alan Henderson,PF,6-9,235,1972-12-02,us,5,Indiana
6,2001,ATL,Jim Jackson,SF,6-6,220,1970-10-14,us,8,Ohio State
7,2001,ATL,Anthony Johnson,PG,6-3,190,1974-10-02,us,3,College of Charleston
8,2001,ATL,DerMarr Johnson,SF,6-9,201,1980-05-05,us,R,Cincinnati
9,2001,ATL,Brevin Knight,PG,5-10,173,1975-11-08,us,3,Stanford


In [207]:
# Get date of start season
schedule = pd.read_csv('./pipeline_output/schedules_training_dataset_2022-03-30.csv')
schedule.head()

,id_season,tm,game_date,time_start,extdom,opponent,w_l,overtime,pts_tm,pts_opp,w_tot,l_tot,streak_w_l
0,2001,ATL,2000-10-31,7:30p,NaN,Charlotte Hornets,L,NaN,82.0,106.0,0.0,1.0,L 1
1,2001,ATL,2000-11-02,7:30p,@,New York Knicks,L,NaN,69.0,94.0,0.0,2.0,L 2
2,2001,ATL,2000-11-04,7:30p,NaN,Orlando Magic,L,NaN,104.0,107.0,0.0,3.0,L 3
3,2001,ATL,2000-11-06,10:00p,@,Vancouver Grizzlies,L,NaN,87.0,97.0,0.0,4.0,L 4
4,2001,ATL,2000-11-07,10:00p,@,Portland Trail Blazers,L,NaN,88.0,97.0,0.0,5.0,L 5


In [208]:
gb = schedule.groupby(['id_season'])
min_date_season = gb.agg({'game_date' : np.min}).reset_index()


In [209]:
players_attributes = pd.merge(
    players_attributes,
    min_date_season,
    how='left',
    left_on=['id_season'],
    right_on=['id_season'])

players_attributes.head()

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College,game_date
0,2001,ATL,Cal Bowdler,SF,6-10,245,1977-03-31,us,1,Old Dominion,2000-10-31
1,2001,ATL,Ira Bowman,PG,6-5,195,1973-06-11,us,1,"Providence, Penn",2000-10-31
2,2001,ATL,Sean Colson,PG,6-0,175,1975-07-01,us,R,"Rhode Island, UNC Charlotte",2000-10-31
3,2001,ATL,Chris Crawford,PF,6-9,235,1975-05-13,us,3,Marquette,2000-10-31
4,2001,ATL,Dion Glover,SG,6-5,228,1978-10-22,us,1,Georgia Tech,2000-10-31


In [210]:
# Age at the start of the season
players_attributes['Age'] = (pd.to_datetime(players_attributes['game_date']) - pd.to_datetime(players_attributes['BirthDate'])).dt.days
players_attributes['Age'] = players_attributes['Age'] / 365
players_attributes['Age'] = pd.to_numeric(players_attributes['Age']).round(2)
players_attributes.head()

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College,game_date,Age
0,2001,ATL,Cal Bowdler,SF,6-10,245,1977-03-31,us,1,Old Dominion,2000-10-31,23.60
1,2001,ATL,Ira Bowman,PG,6-5,195,1973-06-11,us,1,"Providence, Penn",2000-10-31,27.41
2,2001,ATL,Sean Colson,PG,6-0,175,1975-07-01,us,R,"Rhode Island, UNC Charlotte",2000-10-31,25.35
3,2001,ATL,Chris Crawford,PF,6-9,235,1975-05-13,us,3,Marquette,2000-10-31,25.49
4,2001,ATL,Dion Glover,SG,6-5,228,1978-10-22,us,1,Georgia Tech,2000-10-31,22.04


In [211]:
#------------------------
# Rename Rookie

players_attributes['Experience'] = pd.to_numeric(np.where(players_attributes['Experience'] == 'R', 0, players_attributes['Experience']  ))
players_attributes.head()

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College,game_date,Age
0,2001,ATL,Cal Bowdler,SF,6-10,245,1977-03-31,us,1,Old Dominion,2000-10-31,23.60
1,2001,ATL,Ira Bowman,PG,6-5,195,1973-06-11,us,1,"Providence, Penn",2000-10-31,27.41
2,2001,ATL,Sean Colson,PG,6-0,175,1975-07-01,us,0,"Rhode Island, UNC Charlotte",2000-10-31,25.35
3,2001,ATL,Chris Crawford,PF,6-9,235,1975-05-13,us,3,Marquette,2000-10-31,25.49
4,2001,ATL,Dion Glover,SG,6-5,228,1978-10-22,us,1,Georgia Tech,2000-10-31,22.04


In [212]:
#------------------------
# Convert Ht from us sizu to cm size

players_attributes[['foot','inch']] = players_attributes['Ht'].str.split('-', expand=True)
players_attributes['cm_size'] = pd.to_numeric(players_attributes['foot']) * 30.48 + pd.to_numeric(players_attributes['inch']) * 2.54
players_attributes.head()

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College,game_date,Age,foot,inch,cm_size
0,2001,ATL,Cal Bowdler,SF,6-10,245,1977-03-31,us,1,Old Dominion,2000-10-31,23.60,6,10,208.28
1,2001,ATL,Ira Bowman,PG,6-5,195,1973-06-11,us,1,"Providence, Penn",2000-10-31,27.41,6,5,195.58
2,2001,ATL,Sean Colson,PG,6-0,175,1975-07-01,us,0,"Rhode Island, UNC Charlotte",2000-10-31,25.35,6,0,182.88
3,2001,ATL,Chris Crawford,PF,6-9,235,1975-05-13,us,3,Marquette,2000-10-31,25.49,6,9,205.74
4,2001,ATL,Dion Glover,SG,6-5,228,1978-10-22,us,1,Georgia Tech,2000-10-31,22.04,6,5,195.58


In [213]:
# Weight to numeric
players_attributes['Wt'] = pd.to_numeric(players_attributes['Wt'])

In [214]:
#-------------------------------
# Add salary

players_salary = pd.read_csv('./pipeline_output/nba_players_salaries_2022-09-12.csv')
players_salary.head()

,RK,NAME,TEAM,SALARY,year,name,salary
0,1,"Kevin Garnett, PF",Minnesota Timberwolves,"$19,600,000",2001,Kevin Garnett,19600000.0
1,2,"Shaquille O'Neal, C",Los Angeles Lakers,"$19,285,000",2001,Shaquille O'Neal,19285000.0
2,3,"Alonzo Mourning, C",Miami Heat,"$16,879,000",2001,Alonzo Mourning,16879000.0
3,4,"Juwan Howard, PF",Dallas Mavericks,"$16,875,000",2001,Juwan Howard,16875000.0
4,5,"Hakeem Olajuwon, C",Houston Rockets,"$16,685,000",2001,Hakeem Olajuwon,16685000.0


In [215]:
players_salary['name'] = players_salary['name'].str.replace("'", '', regex=True)
players_salary['name'] = players_salary['name'].str.replace("-", '', regex=True)
players_salary['name'] = players_salary['name'].str.replace(".", '', regex=True)
players_salary['name'] = players_salary['name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

players_salary['salary'] = players_salary['salary'] / 10**6

In [216]:
players_salary.head()

,RK,NAME,TEAM,SALARY,year,name,salary
0,1,"Kevin Garnett, PF",Minnesota Timberwolves,"$19,600,000",2001,Kevin Garnett,19.600
1,2,"Shaquille O'Neal, C",Los Angeles Lakers,"$19,285,000",2001,Shaquille ONeal,19.285
2,3,"Alonzo Mourning, C",Miami Heat,"$16,879,000",2001,Alonzo Mourning,16.879
3,4,"Juwan Howard, PF",Dallas Mavericks,"$16,875,000",2001,Juwan Howard,16.875
4,5,"Hakeem Olajuwon, C",Houston Rockets,"$16,685,000",2001,Hakeem Olajuwon,16.685


In [217]:
players_attributes = pd.merge(
    players_attributes,
    players_salary[['name','year','salary']],
    how='left',
    left_on=['Name', 'id_season'],
    right_on=['name', 'year'])

players_attributes = players_attributes[['id_season', 'tm', 'Name', 'Position', 'Wt', 'Experience', 'Age', 'cm_size', 'salary']]
players_attributes.head(10)

,id_season,tm,Name,Position,Wt,Experience,Age,cm_size,salary
0,2001,ATL,Cal Bowdler,SF,245,1,23.60,208.28,1.102
1,2001,ATL,Ira Bowman,PG,195,1,27.41,195.58,NaN
2,2001,ATL,Sean Colson,PG,175,0,25.35,182.88,0.083
3,2001,ATL,Chris Crawford,PF,235,3,25.49,205.74,2.250
4,2001,ATL,Dion Glover,SG,228,1,22.04,195.58,0.960
5,2001,ATL,Alan Henderson,PF,235,5,27.93,205.74,5.909
6,2001,ATL,Jim Jackson,SF,220,8,30.07,198.12,NaN
7,2001,ATL,Anthony Johnson,PG,190,3,26.10,190.50,1.200
8,2001,ATL,DerMarr Johnson,SF,201,0,20.50,205.74,2.107
9,2001,ATL,Brevin Knight,PG,173,3,25.00,177.80,4.250


In [218]:
test = players_attributes[players_attributes['salary'].isna()]
test.shape

(1833, 9)

In [192]:
test.head()

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College,game_date,Age,foot,inch,cm_size,name,year,salary
1,2001,ATL,Ira Bowman,PG,6-5,195,1973-06-11,us,1,"Providence, Penn",2000-10-31,27.41,6,5,195.58,NaN,NaN,NaN
6,2001,ATL,Jim Jackson,SF,6-6,220,1970-10-14,us,8,Ohio State,2000-10-31,30.07,6,6,198.12,NaN,NaN,NaN
13,2001,ATL,Anthony Miller,PF,6-9,225,1971-10-22,us,6,Michigan State,2000-10-31,29.05,6,9,205.74,NaN,NaN,NaN
17,2001,ATL,Andy Panko,SF,6-9,222,1977-11-27,us,0,Lebanon Valley College,2000-10-31,22.94,6,9,205.74,NaN,NaN,NaN
18,2001,ATL,Larry Robinson,PG,6-3,180,1968-01-11,us,5,Centenary (LA),2000-10-31,32.83,6,3,190.50,NaN,NaN,NaN
19,2001,ATL,Pepe Sanchez,PG,6-4,195,1977-05-08,ar,0,Temple,2000-10-31,23.50,6,4,193.04,NaN,NaN,NaN
20,2001,ATL,Tony Smith,SG,6-3,185,1968-06-14,us,8,Marquette,2000-10-31,32.40,6,3,190.50,NaN,NaN,NaN
28,2001,BOS,Chris Carr,SG,6-5,207,1974-03-12,us,5,Southern Illinois,2000-10-31,26.66,6,5,195.58,NaN,NaN,NaN
49,2001,CHH,Lee Nailon,SF,6-9,238,1975-02-22,us,0,"Butler County Community College, TCU",2000-10-31,25.71,6,9,205.74,NaN,NaN,NaN
52,2001,CHH,Terrance Roberson,SF,6-7,215,1976-12-30,us,0,Fresno State,2000-10-31,23.85,6,7,200.66,NaN,NaN,NaN


In [ ]:

# ####################################################################################################################
# #
# #
# # FEATURE ENGINEERING:
# #
# #
# ####################################################################################################################

# #-----------------------------------------
# # wt - NOTHING TO DO - WEIGHT EXPRESS IN LBS
# #-----------------------------------------

# data_player_info_and_salary_full$wt <- as.numeric(data_player_info_and_salary_full$wt)
# summary(data_player_info_and_salary_full$wt)

# #-----------------------------------------
# # birth_date
# #-----------------------------------------

# # FROMAT DATE = January 14, 1985 SO WE CONVERT TO as.Date
# data_player_info_and_salary_full$birth_date_2 <- as.Date(data_player_info_and_salary_full$birth_date, "%B %d, %Y")

# # TO HAVE THE AGE AT THE START OF THE SEASON WE HAVE TO GET THE DATE FOR THE START SEASON FOR EACH YEAR

# mydb <- dbConnect(MySQL(),
#                   user = 'root',
#                   password = 'P21yfuobi',
#                   dbname= "nba",
#                   host = '127.0.0.1',
#                   port = 3306)


# nba_game_data <- as.data.frame((dbGetQuery(mydb, "SELECT * FROM nba_game_data")))
# data_min_season <- nba_game_data %>% group_by(id_season) %>% summarise(date_start_season = min(as.Date(game_date)))

# data_player_info_and_salary_full <- merge(data_player_info_and_salary_full,
#                                           data_min_season,
#                                           by.x = "year",
#                                           by.y = "id_season",
#                                           all.x = TRUE)



# # GET THE YEAR DIFFERENCE
# data_player_info_and_salary_full$age <- round(as.numeric(difftime(data_player_info_and_salary_full$date_start_season,
#                                                                   data_player_info_and_salary_full$birth_date_2,
#                                                                   unit="weeks"))/52.25,1)



# #-----------------------------------------
# # exp
# #-----------------------------------------

# # WE SET THE FORMAT AS NUMERIC AND RECODE THE R EQUAL TO 0
# data_player_info_and_salary_full$exp <- as.numeric(ifelse(data_player_info_and_salary_full$exp  == "R",
#                                                           0,
#                                                           data_player_info_and_salary_full$exp ))

# #-----------------------------------------
# # ht - 6-5 to 185 cm
# #-----------------------------------------

# # STEP 1 : ISOLATE FOOT WICH IS THE FIRST NUMBER  THEN ISOLATE INCH WICH IS THE SECOND NUMBER
# # 1 inche = 2.54cm
# # 1 foot = 30.48cm

# library(tidyr)
# data_player_info_and_salary_full <- data_player_info_and_salary_full %>% separate(ht,
#                                                                                   c("foot", "inch" ),
#                                                                                   "-")


# data_player_info_and_salary_full$foot <- as.numeric(data_player_info_and_salary_full$foot)
# data_player_info_and_salary_full$inch <- as.numeric(data_player_info_and_salary_full$inch)

# data_player_info_and_salary_full$cm_size <- data_player_info_and_salary_full$foot * 30.48 + data_player_info_and_salary_full$inch * 2.54

# #-----------------------------------------
# # MERGE SALARY NA FILES PROVIDE BY ANTHONYC AND FLORIANC
# #-----------------------------------------


# #-----------------------------------------
# # EXPRESS SALARY IN MILLION
# #-----------------------------------------

# data_player_info_and_salary_full$salary <- data_player_info_and_salary_full$salary / 10^6

# #-----------------------------------------
# # SELECT THE RELEVATN VARAIBLES
# #-----------------------------------------

# # WITH DID THE RECODE TO ASSURE THE DATA MANAGEMENT FOR ALL THE TAB
# data_player_info_and_salary_full$id_season <- data_player_info_and_salary_full$year 

# data_player_info_and_salary_full <- data_player_info_and_salary_full %>% select(player,
#                                                                                 tm,
#                                                                                 id_season,
#                                                                                 pos,
#                                                                                 country,
#                                                                                 college,
#                                                                                 age,
#                                                                                 exp,
#                                                                                 cm_size,
#                                                                                 wt,
#                                                                                 salary)

# #--------------------------------------------------------------------------------
# # SAVE THE FILES
# #-------------------------------------------------------------------------------

# write.csv(data_player_info_and_salary_full,
#           paste0("NBA DEV/NBA PLAYER INFORMATION/data_player_info_and_salary_full_prod.csv"),row.names=FALSE)